In [4]:
# 4. Second Turn: Continue the Thread

question_2 = "How can I filter and view this specific conversation in the Langsmith UI?"

# Pass the SAME unique_thread_id in langsmith_extra metadata
ai_answer_2 = langsmith_conversational_rag(
    question=question_2, 
    langsmith_extra={"metadata": {"conversation_id": unique_thread_id}}
)

print(f"\n--- Question 2 ---\n{question_2}")
print(f"\n--- Answer 2 ---\n{ai_answer_2}")

print("\n\nTwo traces were created with the same 'conversation_id'. Check Langsmith!")


--- Question 2 ---
How can I filter and view this specific conversation in the Langsmith UI?

--- Answer 2 ---
I don't know how to filter and view a specific conversation in the Langsmith UI. The provided context only discusses the @traceable decorator and adding metadata to traces, but not filtering or viewing conversations. It does not provide information on using the Langsmith UI for this purpose.


Two traces were created with the same 'conversation_id'. Check Langsmith!


In [3]:
# 3. First Turn: Start the Conversation Thread (TWEAK)

question_1 = "How do I trace my conversation history using Langsmith?"

# TWEAK: Pass the unique_thread_id in langsmith_extra metadata
ai_answer_1 = langsmith_conversational_rag(
    question=question_1, 
    langsmith_extra={"metadata": {"conversation_id": unique_thread_id}}
)

print(f"--- Question 1 ---\n{question_1}")
print(f"\n--- Answer 1 ---\n{ai_answer_1}")

--- Question 1 ---
How do I trace my conversation history using Langsmith?

--- Answer 1 ---
To trace your conversation history, you can use the @traceable decorator on the relevant functions in LangSmith. This decorator will automatically log inputs, outputs, and errors, providing a record of your conversation history. Additionally, you can add custom metadata to the traces using the 'metadata' dictionary or 'langsmith_extra' parameter for more detailed tracking.


In [2]:
# 2. RAG Pipeline Functions

@traceable(run_type="retriever", name="Retrieve Documents")
def retrieve_documents(question: str):
    # This invokes the retriever defined in utils.py
    return retriever.invoke(question)   

@traceable(run_type="llm", name="Final LLM Call (Groq)")
def call_llm(messages: List[dict]):
    # This LLM call is automatically traced by LangChain/Groq
    return llm_client.invoke(messages)

@traceable(run_type="chain", name="RAG Response Generation")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}
    ]
    response = call_llm(messages)
    return response

@traceable(run_type="chain", name="Root Conversational RAG")
def langsmith_conversational_rag(question: str, **kwargs):
    # kwargs will contain the langsmith_extra metadata
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content

In [1]:
# 1. Setup Environment, Imports, and UUID
import os
from dotenv import load_dotenv
from langsmith import traceable 
from langchain_groq import ChatGroq 
from typing import List
import nest_asyncio
import uuid
from utils import get_vector_db_retriever, RAG_SYSTEM_PROMPT

# --- Configuration ---
load_dotenv()
MODEL_NAME = "llama-3.3-70b-versatile" # Using the confirmed, working Groq model
llm_client = ChatGroq(model=MODEL_NAME)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TWEAK: Generate a unique thread ID for the entire conversation
unique_thread_id = str(uuid.uuid4())
print(f"Generated Unique Thread ID: {unique_thread_id}")

c:\Users\Raghav Gupta\anaconda3\envs\langsmith_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Raghav Gupta\OneDrive\Desktop\Langsmith-MAT496\utils.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Generated Unique Thread ID: 9c88833a-c345-4589-a5a1-0e7670169ab9
